This is a new notebook is to test the new dataset received from Atiq.

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import leafmap
import geemap
import ee

In [40]:
raster_image = "Dir_Mauza.tif"
vector_path = 'new_DataSet-V1.geojson'
gdf = gpd.read_file(vector_path)

In [34]:
gdf.head()

,OBJECTID,Mouza_Name,Land_Use,District,Tehsil,Khasra_No,Area,LULC_Landuse,Area_sqft,Area_Kanal,Shape_Length,Shape_Area,geometry
0,1,Thin Dodag,غيرممكن آبادی,Lower Dir,Adlinzai,,25609.650485,Builtup,25612,4.703765,196.123539,2379.223900,"POLYGON ((72.03524 34.70202, 72.03479 34.70231..."
1,2,Thin Dodag,زرعی اراضی,Lower Dir,Adlinzai,,74031.856150,Agriculture,74038,13.597429,383.002630,6877.812004,"POLYGON ((72.03494 34.70169, 72.03608 34.7009,..."
2,3,Thin Dodag,زرعی اراضی,Lower Dir,Adlinzai,,74484.593450,Agriculture,74492,13.680808,447.389557,6919.872844,"POLYGON ((72.03838 34.70369, 72.03869 34.70359..."
3,4,Thin Dodag,زرعی اراضی,Lower Dir,Adlinzai,,70885.494912,Agriculture,70892,13.019651,371.709293,6585.504311,"POLYGON ((72.03772 34.70422, 72.03886 34.70387..."
4,5,Thin Dodag,زرعی اراضی,Lower Dir,Adlinzai,,96568.890168,Agriculture,96578,17.737006,445.406563,8971.579352,"POLYGON ((72.03923 34.70428, 72.03935 34.70434..."


In [35]:
print(f"Length of dataset{len(gdf)}")
gdf['LULC_Landuse'].groupby(gdf['LULC_Landuse']).count().sort_values(ascending=False)

Length of dataset785


LULC_Landuse
Agriculture    453
Builtup        327
Water body       3
Barren Land      2
Name: LULC_Landuse, dtype: int64

In [36]:
agri = gdf[gdf['LULC_Landuse'] == 'Agriculture']
builtup = gdf[gdf['LULC_Landuse'] == 'Builtup']
print(f"Agriculture: {len(agri)}, Built-up: {len(builtup)}")

Agriculture: 453, Built-up: 327


In [65]:
import leafmap

vis = {'bands': [3,2,1], 'min': 0, 'max': 4000,
       'gamma': 1.3
       }

m = leafmap.Map()
m.add_basemap("Satellite")

# ✅ Proper syntax with viz_params
m.add_raster(raster_image, layer_name='Satellite Image', viz_params=vis, opacity=1.0, nodata=0)

m.add_gdf(agri, layer_name='Agriculture', color='green', opacity=0.2, zoom_to_layer=True)
m.add_gdf(builtup, layer_name='Built-up', color='red', opacity=0.2)

m.add_layer_manager()
m


Map(center=[34.7041205, 72.0423705], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…

In [66]:
import geoai

In [ ]:
geoai.view_raster(raster_image, vis_params={'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, layer_name='Sentinel-2 Image')

In [72]:
boundary = geemap.gdf_to_ee(gdf)

In [ ]:

collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2025-01-01', '2025-06-30').filterBounds(boundary.geometry()).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10));

selected_image = collection.first().clip(boundary.geometry())
m = geemap.Map()
vis = {
    'min': 0,
    'max': 3000,    
    'bands': ['B4', 'B3', 'B2'],
      
}
m.add_layer(selected_image,vis,'Sentinel 2')
m.centerObject(selected_image,8)
m



Map(center=[34.704858881289006, 72.04185225577189], controls=(WidgetControl(options=['position', 'transparent_…

# Downloading the image

In [15]:
# Lets Convert the gdf of the original file to ee features utilizing earth engine power
ee.Authenticate()

ee.Initialize()

In [16]:
boundary = geemap.gdf_to_ee(gdf) # this is for cliping the images

In [18]:
# Define a function to get imagery an apply filter
def get_image (start_date,end_date):
    collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    filtered_collection = collection.filterDate(start_date, end_date).filterBounds(boundary.geometry()).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
    selected_image = filtered_collection.first()
    return selected_image

In [19]:
raster_image = get_image('2025-01-01', '2025-06-30')

In [ ]:
# Lets Download the image locally.
# Define export parameters
export_params = {
  'image': raster_image.clip(boundary.geometry()),
  'description': 'Sentinel2_Export',
  'scale': 10,
  'region': boundary.geometry(),
  'fileFormat': 'GeoTIFF',
  'formatOptions': {
    'cloudOptimized': True  # 👈 This is not a true COG, but GEE accepts it for internal optimization
  },
  'maxPixels': 1e13
}

# Export to Google Drive
Export.image.toDrive(**export_params)


In [27]:
task = ee.batch.Export.image.toDrive(**export_params)
task.start()

import time
while task.active():
    print('Polling for task (id: {}).'.format(task.id))
    time.sleep(10)

print('Task status:', task.status())

EEException: Request had insufficient authentication scopes.

In [23]:
m = leafmap.Map()
m.add_basemap("Satellite")
m.add_gdf(agri, layer_name='Agriculture', color='green', opacity=0.2, zoom_to_layer=True)
m.add_gdf(builtup, layer_name='Built-up', color='red', opacity=0.2)
m.add_ee_layer(raster_image, {}, 'Satellite Image')
m.add_layer_control()
m

The provided EE tile layer Satellite Image does not exist.


Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…